In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Define the cryptocurrency symbol and the time frame
symbol = 'BTC-USD'
start_date = '2017-01-01'

In [ ]:
# Download the data from Yahoo Finance API
data = yf.download(symbol, start_date)

In [ ]:
# Preprocess the data
data = data[['Close']]
data.dropna(inplace=True)

In [ ]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

In [ ]:
# Create the training data set
train_data = scaled_data[:int(len(data)*0.8), :]

In [ ]:
# Split the training data set into x_train and y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Reshape the data for LSTM model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
# Define the LSTM model
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32)

In [ ]:
# Create the testing data set
test_data = scaled_data[int(len(data)*0.8)-60:, :]

In [ ]:
# Split the testing data set into x_test and y_test
x_test = []
y_test = data[int(len(data)*0.8):]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)

In [ ]:
# Reshape the data for LSTM model
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Make predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# Evaluate the model
rmse = np.sqrt(np.mean(((predictions - y_test)**2)))
print('Root Mean Squared Error:', rmse)